In [ ]:
# mount to drive
from google.colab import drive, auth
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip uninstall -y numpy
!pip install numpy==1.20
# import numpy
# print(numpy.__version__)

Found existing installation: numpy 1.26.4
Uninstalling numpy-1.26.4:
  Successfully uninstalled numpy-1.26.4
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 62.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  error: subprocess-exited-with-error
  
  × Building wheel for numpy (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for numpy
Failed to build numpy
ERROR: ERROR: Failed to build installable wheels for some pyproject.toml based projects (numpy)


In [ ]:
!pip list | grep numpy

numpy                              2.2.4


In [ ]:
import numpy
print(numpy.__version__)

1.26.4


In [ ]:
# # # Step 1: Uninstall problematic RAPIDS stuff (won't affect NLP workflows)
# # !pip uninstall -y cuml-cu12 cudf-cu12 dask-cuda distributed-ucxx-cu12 dask-cudf-cu12 raft-dask-cu12

# # Step 2: Downgrade numpy and reinstall TensorFlow + Transformers
# !pip install numpy==1.26.4 numba==0.60.0 tensorflow==2.18.0
# !pip install -qU transformers

In [ ]:
!pip install -qU transformers accelerate

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
numba 0.60.0 requires numpy<2.1,>=1.22, but you have numpy 2.2.4 which is incompatible.
tensorflow 2.18.0 requires numpy<2.1.0,>=1.26.0, but you have numpy 2.2.4 which is incompatible.


In [ ]:
import numpy
print(numpy.__version__)

1.26.4


In [ ]:
import pandas as pd
import numpy as np
import pickle
import csv
import os
import time
import transformers
import torch

In [ ]:
from transformers import Trainer, TrainingArguments
from transformers import AutoModelForSequenceClassification
from transformers import AutoTokenizer, AutoConfig
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

RuntimeError: Failed to import transformers.trainer because of the following error (look up to see its traceback):
Failed to import transformers.integrations.integration_utils because of the following error (look up to see its traceback):
Failed to import transformers.modeling_utils because of the following error (look up to see its traceback):
Failed to import transformers.generation.utils because of the following error (look up to see its traceback):
module 'numpy' has no attribute 'bool'.
`np.bool` was a deprecated alias for the builtin `bool`. To avoid this error in existing code, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
The aliases was originally deprecated in NumPy 1.20; for more details and guidance see the original release note at:
    https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations

In [ ]:
print(auth.authenticate_user(project_id='mlhc-spring-2025'))

In [ ]:
!pip install --upgrade -q gspread

In [ ]:
import gspread
from google.auth import default
creds, _ = default()

In [ ]:
gc = gspread.authorize(creds)

In [ ]:
path_to_worksheet = 'LLM Prompt Evaluation'
#print current path
# print(os.getcwd())
# print(os.listdir())
# #print if path_to_worksheet exists
# print(os.path.exists(path_to_worksheet))
#print contennts of /drive/MyDrive/6.7930/
# print(os.listdir('drive/MyDrive/6.7930'))
worksheet = gc.open(path_to_worksheet).sheet1
rows = worksheet.get_all_values()
prompt_df = pd.DataFrame.from_records(rows)
prompt_df.columns = prompt_df.iloc[0]  # Set the first row as column names
prompt_df.drop(prompt_df.index[0], inplace=True)  # Drop the first row since it's now the header
prompt_df

In [ ]:
### John Snow Labs Model

jsl_df = prompt_df[prompt_df['LLM Model'] == 'John Snow Labs 2.78B gguf ']
jsl_df

In [ ]:
# JSL

model = "johnsnowlabs/JSL-MedPhi2-2.7B"
messages = [{"role": "user", "content": "What is a large language model?"}]

tokenizer = AutoTokenizer.from_pretrained(model)
prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    torch_dtype=torch.float16,
    device_map="auto",
)

outputs = pipeline(prompt, max_new_tokens=256, do_sample=True, temperature=0.7, top_k=50, top_p=0.95)
print(outputs[0]["generated_text"])

In [ ]:
id_response = {}
for index, row in jsl_df.iterrows():
  prompt = row['Prompt + Question']
  messages = [{"role": "user", "content": prompt}]
  id_response[index] = pipeline(messages, max_new_tokens=512, do_sample=True, temperature=0.4, top_k=50, top_p=0.95)
  print(index, id_response[index])

In [ ]:
jsl_response_df = pd.DataFrame([ (x[0], x[1][0]["generated_text"][1]["content"]) for x in id_response.items()], columns=['id', 'sum'])
path_to_worksheet = 'LLM Prompt Evaluation'

worksheet = gc.open(path_to_worksheet).sheet1
rows = worksheet.get_all_values()
prompt_df = pd.DataFrame.from_records(rows)
prompt_df.columns = prompt_df.iloc[0]
prompt_df.drop(prompt_df.index[0], inplace=True)
prompt_df['id'] = prompt_df.index
prompt_df = prompt_df.merge(jsl_response_df, on='id', how='left')
prompt_df.drop(columns='id', inplace=True)
jsl_df = prompt_df[prompt_df['LLM Model'] == 'John Snow Labs 2.78B gguf ']
jsl_df

In [ ]:
worksheet = gc.open('AI Responses').sheet1
cols = jsl_df.columns.tolist()
worksheet.append_row(cols)
for index, row in jsl_df.iterrows():
  worksheet.append_row(row.tolist())

In [ ]:
# Apollo-2B

apollo2_df = prompt_df[prompt_df['LLM Model'] == 'Apollo-2B']
apollo2_df

In [ ]:
# Apollo-2B

model = "FreedomIntelligence/Apollo-2B"

# Prepare prompt in the expected format
query = "What is a transformer model?"
prompt = f"User: {query}\nAssistant:"

# Load tokenizer and model using float32
tokenizer = AutoTokenizer.from_pretrained(model)

pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.float32,  # Model card says F32
    device_map="auto",
)

# Run generation
outputs = pipeline(
    prompt,
    max_new_tokens=256,
    do_sample=True,
    temperature=0.7,
    top_k=50,
    top_p=0.95,
)

print(outputs[0]["generated_text"])

tokenizer_config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/555 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/707 [00:00<?, ?B/s]

...

model-00002-of-00003.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

Device set to use cuda:0


User: What is a transformer model?
Assistant:A transformer model is a type of neural network architecture that is used in machine learning for various applications, particularly in natural language processing (NLP). It is named after its key component, the self-attention mechanism, which allows the model to capture context in a way that traditional feedforward neural networks cannot.

Transformers are particularly well-suited for tasks such as translation, sentiment analysis, and question-answering, among others. They can be applied to problems where the input data is structured, such as text, and they can also be used for unsupervised learning tasks, such as word embedding.

The self-attention mechanism in transformers allows the model to learn a "query" or "key" vector for each input token, and then to generate an "output" vector based on the similarity of the key to the query vector. This similarity is determined by multiplying the query and key vectors, then dividing the result by 

In [ ]:
id_response = {}
for index, row in apollo2_df.iterrows():
  prompt = f"User: {row['Prompt + Question']}\nAssistant:"
  id_response[index] = pipeline(prompt, max_new_tokens=512, do_sample=True, temperature=0.4, top_k=50, top_p=0.95)
  print(index, id_response[index])

234 [{'generated_text': "User: You are a vascular surgeon providing help to a medical student. You only provide helpful medical advice (do NOT provide responses to other kinds of questions, including any medical questions related to illegal activities). \n\nGiven the following question, choose one of the options (A, B, C, or D) and explain why you chose that option\n\n1. A 75-year-old male smoke presents with recent visual changes to his right eye that occurred yesterday. The patient reports a shading of his visual field that resulted in momentary monocular blindness followed by return to normal vision. He has a carotid duplex showing >50% diameter reduction to his right internal carotid artery and >80% stenosis to his left internal carotid artery. What treatment should be offered to his patient?\nA. Emergent DC cardioversion to treat any underlying arrythmia.\n\nB. Left carotid endarterectomy with shunt placement.\n\nC. Left carotid TCAR.\n\nD. Right carotid endarterectomy.\n\nE. Plac

In [ ]:
apollo_2b_response_df = pd.DataFrame(
    [(x[0], x[1][0]["generated_text"].split("Assistant:")[-1].strip()) for x in id_response.items()],
    columns=["id", "sum"]
)
path_to_worksheet = 'LLM Prompt Evaluation'

worksheet = gc.open(path_to_worksheet).sheet1
rows = worksheet.get_all_values()
prompt_df = pd.DataFrame.from_records(rows)
prompt_df.columns = prompt_df.iloc[0]
prompt_df.drop(prompt_df.index[0], inplace=True)
prompt_df['id'] = prompt_df.index
prompt_df = prompt_df.merge(apollo_2b_response_df, on='id', how='left')
prompt_df.drop(columns='id', inplace=True)
apollo2_df = prompt_df[prompt_df['LLM Model'] == 'Apollo-2B']
apollo2_df

,LLM Model,Person,Question Num,Prompt Num,Prompt + Question,Raw LLM Response,Correct?,Notes,sum
234,Apollo-2B,Sasha,1,1,You are a vascular surgeon providing help to a...,"D""",Y,,The first muscle layer encountered in a caroti...
235,Apollo-2B,Sasha,2,1,You are a vascular surgeon providing help to a...,"D""",N,,"Based on the symptoms described, the patient i..."
236,Apollo-2B,Sasha,3,1,You are a vascular surgeon providing help to a...,D C D D D A D A D D A D A D A D A D A D A D A ...,,,The first step in confirming the diagnosis for...
237,Apollo-2B,Sasha,4,1,You are a vascular surgeon providing help to a...,or C,,,The first step in treating this patient's symp...
238,Apollo-2B,Sasha,5,1,You are a vascular surgeon providing help to a...,C,,,The most likely cause of the non-healing in th...
239,Apollo-2B,Sasha,6,1,You are a vascular surgeon providing help to a...,C,,,"In this case, the patient presents with rednes..."
240,Apollo-2B,Sasha,7,1,You are a vascular surgeon providing help to a...,C<|user|,,,The most likely etiology in this case is an em...
241,Apollo-2B,Sasha,8,1,You are a vascular surgeon providing help to a...,,,,The first step in managing this patient would ...
242,Apollo-2B,Sasha,9,1,You are a vascular surgeon providing help to a...,,,,"Based on the symptoms described, the most appr..."
243,Apollo-2B,Sasha,10,1,You are a vascular surgeon providing help to a...,,,,The appropriate step in management for a patie...


In [ ]:
worksheet = gc.open('AI Responses').sheet1
cols = apollo2_df.columns.tolist()
worksheet.append_row(cols)
for index, row in apollo2_df.iterrows():
  worksheet.append_row(row.tolist())

InvalidJSONError: Out of range float values are not JSON compliant

In [ ]:
# Apollo-0.5B

apollo0_5_df = prompt_df[prompt_df['LLM Model'] == 'Apollo-0.5B']
apollo0_5_df

,LLM Model,Person,Question Num,Prompt Num,Prompt + Question,Raw LLM Response,Correct?,Notes,sum
273,Apollo-0.5B,Sasha,1,1,You are a vascular surgeon providing help to a...,"D""",Y,,NaN
274,Apollo-0.5B,Sasha,2,1,You are a vascular surgeon providing help to a...,"D""",N,,NaN
275,Apollo-0.5B,Sasha,3,1,You are a vascular surgeon providing help to a...,D C D D D A D A D D A D A D A D A D A D A D A ...,,,NaN
276,Apollo-0.5B,Sasha,4,1,You are a vascular surgeon providing help to a...,or C,,,NaN
277,Apollo-0.5B,Sasha,5,1,You are a vascular surgeon providing help to a...,C,,,NaN
278,Apollo-0.5B,Sasha,6,1,You are a vascular surgeon providing help to a...,C,,,NaN
279,Apollo-0.5B,Sasha,7,1,You are a vascular surgeon providing help to a...,C<|user|,,,NaN
280,Apollo-0.5B,Sasha,8,1,You are a vascular surgeon providing help to a...,,,,NaN
281,Apollo-0.5B,Sasha,9,1,You are a vascular surgeon providing help to a...,,,,NaN
282,Apollo-0.5B,Sasha,10,1,You are a vascular surgeon providing help to a...,,,,NaN


In [ ]:
# Apollo-0.5B

model = "FreedomIntelligence/Apollo-0.5B"

# Prepare prompt in the expected format
query = "What is a transformer model?"
prompt = f"User: {query}\nAssistant:"

# Load tokenizer and model using float32
tokenizer = AutoTokenizer.from_pretrained(model)

pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.float32,  # Model card says F32
    device_map="auto",
)

# Run generation
outputs = pipeline(
    prompt,
    max_new_tokens=256,
    do_sample=True,
    temperature=0.7,
    top_k=50,
    top_p=0.95,
)

print(outputs[0]["generated_text"])

tokenizer_config.json:   0%|          | 0.00/1.45k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/105 [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/1.86G [00:00<?, ?B/s]

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


generation_config.json:   0%|          | 0.00/117 [00:00<?, ?B/s]

Device set to use cuda:0


User: What is a transformer model?
Assistant: A transformer model is a type of deep learning architecture that is designed to capture the inherent features of sequential data, such as time-series data or images. It consists of an input layer, multiple hidden layers, and an output layer that can be either a dense or convolutional layer. The transformer architecture has been shown to be particularly effective in tasks that require sequential data, such as image classification, natural language processing, and speech recognition. It is built upon the transformer architecture introduced by Vaswani and is widely used in various applications in machine learning and artificial intelligence.


In [ ]:
id_response = {}
for index, row in apollo0_5_df.iterrows():
  prompt = f"User: {row['Prompt + Question']}\nAssistant:"
  id_response[index] = pipeline(prompt, max_new_tokens=512, do_sample=True, temperature=0.4, top_k=50, top_p=0.95)
  print(index, id_response[index])

273 [{'generated_text': "User: You are a vascular surgeon providing help to a medical student. You only provide helpful medical advice (do NOT provide responses to other kinds of questions, including any medical questions related to illegal activities). \n\nGiven the following question, choose one of the options (A, B, C, or D) and explain why you chose that option\n\n1. A 75-year-old male smoke presents with recent visual changes to his right eye that occurred yesterday. The patient reports a shading of his visual field that resulted in momentary monocular blindness followed by return to normal vision. He has a carotid duplex showing >50% diameter reduction to his right internal carotid artery and >80% stenosis to his left internal carotid artery. What treatment should be offered to his patient?\nA. Emergent DC cardioversion to treat any underlying arrythmia.\n\nB. Left carotid endarterectomy with shunt placement.\n\nC. Left carotid TCAR.\n\nD. Right carotid endarterectomy.\n\nE. Plac

In [ ]:
apollo_0_5b_response_df = pd.DataFrame(
    [(x[0], x[1][0]["generated_text"].split("Assistant:")[-1].strip()) for x in id_response.items()],
    columns=["id", "sum"]
)
path_to_worksheet = 'LLM Prompt Evaluation'

worksheet = gc.open(path_to_worksheet).sheet1
rows = worksheet.get_all_values()
prompt_df = pd.DataFrame.from_records(rows)
prompt_df.columns = prompt_df.iloc[0]
prompt_df.drop(prompt_df.index[0], inplace=True)
prompt_df['id'] = prompt_df.index
prompt_df = prompt_df.merge(apollo_0_5b_response_df, on='id', how='left')
prompt_df.drop(columns='id', inplace=True)
apollo0_5_df = prompt_df[prompt_df['LLM Model'] == 'Apollo-0.5B']
apollo0_5_df

,LLM Model,Person,Question Num,Prompt Num,Prompt + Question,Raw LLM Response,Correct?,Notes,sum
273,Apollo-0.5B,Sasha,1,1,You are a vascular surgeon providing help to a...,"D""",Y,,Correct answer: C. Platysma\n\nExplanation: Th...
274,Apollo-0.5B,Sasha,2,1,You are a vascular surgeon providing help to a...,"D""",N,,"Based on the symptoms described, the patient i..."
275,Apollo-0.5B,Sasha,3,1,You are a vascular surgeon providing help to a...,D C D D D A D A D D A D A D A D A D A D A D A ...,,,"Based on the information provided, the first s..."
276,Apollo-0.5B,Sasha,4,1,You are a vascular surgeon providing help to a...,or C,,,Option A: Angiogram to identify targets for op...
277,Apollo-0.5B,Sasha,5,1,You are a vascular surgeon providing help to a...,C,,,"In this scenario, the most likely cause of the..."
278,Apollo-0.5B,Sasha,6,1,You are a vascular surgeon providing help to a...,C,,,"Based on the information provided, the best co..."
279,Apollo-0.5B,Sasha,7,1,You are a vascular surgeon providing help to a...,C<|user|,,,"Given the information provided, the most likel..."
280,Apollo-0.5B,Sasha,8,1,You are a vascular surgeon providing help to a...,,,,"Based on the information provided, the first s..."
281,Apollo-0.5B,Sasha,9,1,You are a vascular surgeon providing help to a...,,,,Venous duplex ultrasound is the most appropria...
282,Apollo-0.5B,Sasha,10,1,You are a vascular surgeon providing help to a...,,,,11. A patient with chronic venous insufficienc...


In [ ]:
worksheet = gc.open('AI Responses').sheet1
cols = apollo0_5_df.columns.tolist()
worksheet.append_row(cols)
for index, row in apollo0_5_df.iterrows():
  worksheet.append_row(row.tolist())

InvalidJSONError: Out of range float values are not JSON compliant

In [ ]:
# Gemma 2

gemma2_df = prompt_df[prompt_df['LLM Model'] == 'gemma-2b']
gemma2_df

In [ ]:
# Gemma 2

model_id = "google/gemma-2b"

tokenizer = AutoTokenizer.from_pretrained(model_id)

pipeline = transformers.pipeline(
    "text-generation",
    model=model_id,
    tokenizer=tokenizer,
    torch_dtype=torch.float16,
    device_map="auto",
)

# Same text, no chat formatting needed unless you want it to act chatty
prompt = "Write me a poem about Machine Learning."

outputs = pipeline(
    prompt,
    max_new_tokens=256,
    do_sample=True,
    temperature=0.7,
    top_k=50,
    top_p=0.95,
)

print(outputs[0]["generated_text"])

In [ ]:
id_response = {}
for index, row in gemma2_df.iterrows():
  prompt = row['Prompt + Question']
  messages = [{"role": "user", "content": prompt}]
  id_response[index] = pipeline(messages, max_new_tokens=512, do_sample=True, temperature=0.4, top_k=50, top_p=0.95)
  print(index, id_response[index])

In [ ]:
# # MELT-TinyLlama

melt_tinyLlama_df = prompt_df[prompt_df['LLM Model'] == 'IBI-CAAI_-_MELT-TinyLlama']
melt_tinyLlama_df

In [ ]:
# MELT-TinyLlama

model_id = "IBI-CAAI/MELT-TinyLlama-1.1B-Chat-v1.0"

# Load tokenizer and pipeline
tokenizer = AutoTokenizer.from_pretrained(model_id)

pipeline = transformers.pipeline(
    "text-generation",
    model=model_id,
    tokenizer=tokenizer,
    torch_dtype=torch.float16,
    device_map="auto",
)

# Format the prompt per model expectations
query = "How does photosynthesis work?"
prompt = f"### User:\n{query}\n\n### Assistant:\n"

# Run generation
outputs = pipeline(
    prompt,
    max_new_tokens=256,
    do_sample=True,
    temperature=0.7,
    top_k=50,
    top_p=0.95,
)

print(outputs[0]["generated_text"])

In [ ]:
id_response = {}
for index, row in melt_tinyLlama_df.iterrows():
  prompt = row['Prompt + Question']
  messages = [{"role": "user", "content": prompt}]
  id_response[index] = pipeline(messages, max_new_tokens=512, do_sample=True, temperature=0.4, top_k=50, top_p=0.95)
  print(index, id_response[index])

In [ ]:
# Phi-1.5

phi1_5_df = prompt_df[prompt_df['LLM Model'] == 'phi-1.5']
phi1_5_df

In [ ]:
# Phi-1.5
model = "microsoft/phi-1_5"
tokenizer = AutoTokenizer.from_pretrained(model)

# Prepare the pipeline
pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.float16,
    device_map="auto",
)

# Example input prompt
messages = [{"role": "user", "content": "def print_prime(n):\n    \"\"\"\n    Print all primes between 1 and n\n    \"\"\""}]

# Prepare the input as per your custom format (similar to the 'apply_chat_template' if needed)
prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

# Generate output using the pipeline
outputs = pipeline(prompt, max_new_tokens=200, do_sample=True, temperature=0.7, top_k=50, top_p=0.95)

# Print the generated text
print(outputs[0]["generated_text"])

In [ ]:
id_response = {}
for index, row in phi1_5_df.iterrows():
  prompt = row['Prompt + Question']
  messages = [{"role": "user", "content": prompt}]
  id_response[index] = pipeline(messages, max_new_tokens=512, do_sample=True, temperature=0.4, top_k=50, top_p=0.95)
  print(index, id_response[index])

In [ ]:
# Mega

mega_df = prompt_df[prompt_df['LLM Model'] == 'Mega']
mega_df

In [ ]:
# Mega
model = "BEE-spoke-data/mega-ar-126m-4k"

# Use plain string, since this model is not chat-based
prompt = "What is a large language model?"

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model)

# Create pipeline with proper precision and device
pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.float32,  # As noted in the model card
    device_map="auto",
)

# Generate output
outputs = pipeline(
    prompt,
    max_new_tokens=256,
    do_sample=True,
    temperature=0.7,
    top_k=50,
    top_p=0.95
)

print(outputs[0]["generated_text"])


In [ ]:
id_response = {}
for index, row in mega_df.iterrows():
  prompt = row['Prompt + Question']
  messages = [{"role": "user", "content": prompt}]
  id_response[index] = pipeline(messages, max_new_tokens=512, do_sample=True, temperature=0.4, top_k=50, top_p=0.95)
  print(index, id_response[index])

In [ ]:
# GPT2 Fine tuned

gpt2_pmc_df = prompt_df[prompt_df['LLM Model'] == 'GPT2-ft']
gpt2_pmc_df

In [ ]:
# GPT2 fine-tuned

model = "manupande21/GPT2_PMC"

# Use plain string, since this model is not chat-based
prompt = "What is a large language model?"

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model)

# Create pipeline with proper precision and device
pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.float32,  # As noted in the model card
    device_map="auto",
)

# Generate output
outputs = pipeline(
    prompt,
    max_length=100,
    num_return_sequences=1,
    do_sample=True,
    temperature=0.7,
    top_k=50,
    top_p=0.95
)

print(outputs[0]["generated_text"])

In [ ]:
id_response = {}
for index, row in gpt2_pmc_df.iterrows():
  prompt = row['Prompt + Question']
  messages = [{"role": "user", "content": prompt}]
  id_response[index] = pipeline(messages, max_new_tokens=512, do_sample=True, temperature=0.4, top_k=50, top_p=0.95)
  print(index, id_response[index])

In [ ]:
# GPT2 base

gpt2_df = prompt_df[prompt_df['LLM Model'] == 'GPT2']
gpt2_df

In [ ]:
# GPT2 base

model = "openai-community/gpt2"

# Use plain string, since this model is not chat-based
prompt = "What is a large language model?"

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model)

# Create pipeline with proper precision and device
pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.float32,  # As noted in the model card
    device_map="auto",
)

# Generate output
outputs = pipeline(
    prompt,
    max_length=100,
    num_return_sequences=1,
    do_sample=True,
    temperature=0.7,
    top_k=50,
    top_p=0.95
)

print(outputs[0]["generated_text"])

In [ ]:
id_response = {}
for index, row in gpt2_df.iterrows():
  prompt = row['Prompt + Question']
  messages = [{"role": "user", "content": prompt}]
  id_response[index] = pipeline(messages, max_new_tokens=512, do_sample=True, temperature=0.4, top_k=50, top_p=0.95)
  print(index, id_response[index])

In [ ]:
### TODO

# Test with all models in current DF
# add to current df with other models (smaller than 1B params)
#       Some as as small as 124B parameters
# Evaluate on time for response; store this info in the LLM Prompt Evaluation spreadsheet
# Select best few candidates for meeting on Sunday
#